In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, r2_score

In [2]:
df = pd.read_csv('lic_clean_49_24.csv')

In [3]:
#2021, année covid // interpolate
df.head()
print(df[df["annee"]==2021])
df.loc[df['annee'] == 2021, 'total'] = None
df['total'] = df['total'].interpolate(method='linear')
print(df[df["annee"]==2021])

df=df.drop(df[df['annee']==2024].index)
df['annee'] = df['annee'].apply(str)

    Unnamed: 0  annee   olympique  non-olympique  affinitaire   scolaire  \
72          72   2021  15646920.0      3021926.0    2459656.0  4454662.0   

         total  
72  38518155.0  
    Unnamed: 0  annee   olympique  non-olympique  affinitaire   scolaire  \
72          72   2021  15646920.0      3021926.0    2459656.0  4454662.0   

         total  
72  46481116.5  


In [4]:
df = df.sort_values("annee")
X = df[["annee"]]
y = df[["total"]]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, 
    test_size=0.4, 
    random_state=16
)

print("Années dans X_train :", X_train["annee"].unique())
print("Années dans X_test :", X_test["annee"].unique())

Années dans X_train : <StringArray>
['1961', '1974', '1969', '2013', '1987', '1997', '2016', '1964', '1970',
 '2009', '1995', '2005', '1989', '1966', '1985', '2020', '2006', '2021',
 '1957', '1993', '2000', '1977', '1986', '1949', '1962', '1978', '1963',
 '2012', '1983', '1959', '2019', '1950', '1981', '2001', '1967', '2007',
 '1979', '1965', '2023', '1953', '1980', '2017', '2014', '2018', '1990']
Length: 45, dtype: str
Années dans X_test : <StringArray>
['1976', '1951', '1954', '1991', '2002', '1998', '1996', '2008', '2022',
 '2003', '2011', '1956', '1968', '2004', '1971', '1984', '1952', '1988',
 '2015', '1992', '1955', '1975', '1994', '1999', '1958', '1972', '1960',
 '1973', '2010', '1982']
Length: 30, dtype: str


In [5]:
X_train.head()

,annee
12,1961
25,1974
20,1969
64,2013
38,1987


In [6]:
X.shape

(75, 1)

In [7]:
# date_features=["annee"]

# def dates_transfo(X):
#     print(X.shape)
#     X = pd.DataFrame(X, columns=["annee"])
#     X["annee"] = pd.to_datetime(X["annee"])
    
#     return pd.DataFrame({
#         "annee":  X["annee"].dt.year})

# date_transformer = FunctionTransformer(dates_transfo, validate=False)

# feature_encoder = ColumnTransformer(
#     transformers=[
#        ('date', date_transformer, date_features),
#     ]
# )

categorical_features=["annee"]

categorical_transformer = OneHotEncoder(handle_unknown='ignore')

feature_encoder = ColumnTransformer(
    transformers=[
       ('cat', categorical_transformer, categorical_features),
    ]
)


X_train = feature_encoder.fit_transform(X_train)
X_test = feature_encoder.transform(X_test)

In [8]:
regressor=LinearRegression()
regressor.fit(X_train, y_train)

,"fit_intercept fit_intercept: bool, default=TrueWhether to calculate the intercept for this model. If setto False, no intercept will be used in calculations(i.e. data is expected to be centered).",True
,"copy_X copy_X: bool, default=TrueIf True, X will be copied; else, it may be overwritten.",True
,"tol tol: float, default=1e-6The precision of the solution (`coef_`) is determined by `tol` whichspecifies a different convergence criterion for the `lsqr` solver.`tol` is set as `atol` and `btol` of :func:`scipy.sparse.linalg.lsqr` whenfitting on sparse training data. This parameter has no effect when fittingon dense data... versionadded:: 1.7",1e-06
,"n_jobs n_jobs: int, default=NoneThe number of jobs to use for the computation. This will only providespeedup in case of sufficiently large problems, that is if firstly`n_targets > 1` and secondly `X` is sparse or if `positive` is setto `True`. ``None`` means 1 unless in a:obj:`joblib.parallel_backend` context. ``-1`` means using allprocessors. See :term:`Glossary ` for more details.",None
,"positive positive: bool, default=FalseWhen set to ``True``, forces the coefficients to be positive. Thisoption is only supported for dense arrays.For a comparison between a linear regression model with positive constraintson the regression coefficients and a linear regression without such constraints,see :ref:`sphx_glr_auto_examples_linear_model_plot_nnls.py`... versionadded:: 0.24",False


In [9]:
print(df['annee'].value_counts())

annee
1949    1
1950    1
1951    1
1952    1
1953    1
       ..
2019    1
2020    1
2021    1
2022    1
2023    1
Name: count, Length: 75, dtype: int64


In [10]:
print(X_train.shape)
print(X_train[:5])
print(y_train[:5])

(45, 45)
<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 5 stored elements and shape (5, 45)>
  Coords	Values
  (0, 5)	1.0
  (1, 14)	1.0
  (2, 12)	1.0
  (3, 36)	1.0
  (4, 23)	1.0
         total
12   3223000.0
25   7189000.0
20   4983000.0
64  15901871.0
38  12689000.0


In [16]:
Y_train_pred = regressor.predict(X_train)
Y_test_pred = regressor.predict(X_test)


print(Y_train_pred)
print(Y_test_pred)


[[ 3223000. ]
 [ 7189000. ]
 [ 4983000. ]
 [15901871. ]
 [12689000. ]
 [14047000. ]
 [48652745. ]
 [ 3643000. ]
 [ 5316000. ]
 [15546431. ]
 [14016000. ]
 [14763913. ]
 [12914000. ]
 [ 4060000. ]
 [12114000. ]
 [47140017. ]
 [14801953. ]
 [46481116.5]
 [ 2402000. ]
 [13638000. ]
 [14666000. ]
 [ 8647000. ]
 [12709000. ]
 [ 1935000. ]
 [ 3363000. ]
 [ 8920000. ]
 [ 3385000. ]
 [15636997. ]
 [11456000. ]
 [ 2757000. ]
 [48774267. ]
 [ 2154000. ]
 [10076000. ]
 [14836000. ]
 [ 4493000. ]
 [15101358. ]
 [ 9241000. ]
 [ 3902000. ]
 [49549320. ]
 [ 2252000. ]
 [ 9628000. ]
 [49106777. ]
 [15879293. ]
 [48765914. ]
 [13001000. ]]
[[15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.61111111]
 [15416821.6

In [14]:
Y_test_pred_flat = Y_test_pred.flatten() if len(Y_test_pred.shape) > 1 else Y_test_pred

df_pred = pd.DataFrame({
    "annee": X_test["annee"].values,  
    "total_pred": Y_test_pred_flat
})

print(df_pred.head())

IndexError: Index dimension must be 1 or 2. Got 0